In [9]:
import os
import subprocess
from astropy.table import Table
from astropy.io import ascii

In [13]:
diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
medianDir = '/media/mj1e16/PP AV-TV/medians/'
imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'
sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'

In [26]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='2'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [5]:
def makeConfig(valList,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    

    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test_test.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_test.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    

In [18]:
def findObjects(valList,atList,image,index):
    makeConfig(valList,attributeList=atList)
    subprocess.call(['sex',image,'-c','default_test.sex'])
    tabTot =  ascii.read('/home/mj1e16/keplerPhotometry/sexdiffOutNew2.csv')
    makeDS9RegFile([tabTot],'sextest_{}'.format(index),'sex')

In [16]:
atList = ['DETECT_MINAREA']
valList = [[5],[7],[9],[11],[13],[15]]
image = 'difkplr2011303191211_ffi-cal56.fits'

In [15]:
os.chdir(sexDir)


In [23]:
for x in range(len(valList)):
    findObjects(valList[x],atList,diffDir+image,x)


In [25]:
command = ['ds9']
for x in range(len(valList)):
    command.extend([imageDir+image,'-regions','sextest_{}.reg'.format(x)])
command.extend(['-frame','lock','image'])
print(command)
subprocess.call(command)

['ds9', '/media/mj1e16/PP AV-TV/keplerCal/kplr2010234192745_ffi-cal.fits', '-regions', 'sextest_0.reg', '/media/mj1e16/PP AV-TV/keplerCal/kplr2010234192745_ffi-cal.fits', '-regions', 'sextest_1.reg', '/media/mj1e16/PP AV-TV/keplerCal/kplr2010234192745_ffi-cal.fits', '-regions', 'sextest_2.reg', '/media/mj1e16/PP AV-TV/keplerCal/kplr2010234192745_ffi-cal.fits', '-regions', 'sextest_3.reg', '/media/mj1e16/PP AV-TV/keplerCal/kplr2010234192745_ffi-cal.fits', '-regions', 'sextest_4.reg', '/media/mj1e16/PP AV-TV/keplerCal/kplr2010234192745_ffi-cal.fits', '-regions', 'sextest_5.reg', '-frame', 'lock', 'image']


0